In [1]:
import os
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import joblib

def get_person_activity_range(file_path):
    """
    获取单个npy文件中所有人的活动范围
    返回: (filename, dict) 或 (filename, None)，包含每个body的活动范围信息
    """
    try:
        data = np.load(file_path, allow_pickle=True).item()
        
        # 计算有多少个body
        max_body = 0
        while f"rgb_body{max_body}" in data:
            max_body += 1
        
        file_activity_ranges = {}
        
        # 为每个body计算活动范围
        for body_idx in range(max_body):
            body_key = f"rgb_body{body_idx}"
            if body_key not in data:
                continue
            body_data = data[body_key]
            if body_data.size == 0:
                continue
            
            # 重塑为 (frames*joints, 2)
            points = body_data.reshape(-1, 2)
            # 过滤掉无效点（坐标为0的点）
            valid_points = points[(points[:, 0] != 0) | (points[:, 1] != 0)]
            
            if len(valid_points) > 0:
                # 计算边界
                min_x, min_y = np.min(valid_points, axis=0)
                max_x, max_y = np.max(valid_points, axis=0)
                width = max_x - min_x
                height = max_y - min_y
                
                file_activity_ranges[f"body{body_idx}"] = {
                    'min_x': float(min_x),
                    'min_y': float(min_y), 
                    'max_x': float(max_x),
                    'max_y': float(max_y),
                    'width': float(width),
                    'height': float(height),
                    'skeleton_data': body_data  # 保存原始骨架数据
                }
        
        return os.path.basename(file_path), file_activity_ranges
        
    except Exception as e:
        return os.path.basename(file_path), None

def process_all_skeleton_files(data_dir="nturgb+d_skeletons_npy", max_workers=8):
    """
    使用多线程处理所有骨架文件，获取每个人的活动范围
    
    Parameters:
    - data_dir: 数据目录
    - max_workers: 最大线程数，建议设置为CPU核心数
    """
    # 获取所有文件
    all_files = [f for f in os.listdir(data_dir) if f.endswith(".npy")]
    total_files = len(all_files)
    print(f"总共需要处理 {total_files} 个文件")
    print(f"使用 {max_workers} 个线程进行并行处理")
    
    # 构建完整文件路径
    file_paths = [os.path.join(data_dir, fname) for fname in all_files]
    
    all_activity_ranges = {}
    error_files = []
    total_bodies = 0
    
    # 使用线程池进行并行处理
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 提交所有任务
        future_to_file = {executor.submit(get_person_activity_range, fpath): fpath 
                         for fpath in file_paths}
        
        # 使用tqdm显示进度
        with tqdm(total=total_files, desc="处理骨架文件") as pbar:
            for future in as_completed(future_to_file):
                file_path = future_to_file[future]
                try:
                    filename, file_ranges = future.result()
                    
                    if file_ranges is not None and len(file_ranges) > 0:
                        all_activity_ranges[filename] = file_ranges
                        total_bodies += len(file_ranges)
                    else:
                        error_files.append(filename)
                        
                except Exception as e:
                    filename = os.path.basename(file_path)
                    error_files.append(filename)
                    print(f"处理文件 {filename} 时出错: {e}")
                
                pbar.update(1)
    
    print(f"\n处理完成！")
    print(f"成功处理文件: {len(all_activity_ranges)} 个")
    print(f"检测到的总人数: {total_bodies} 个")
    print(f"错误文件: {len(error_files)} 个")
    
    if error_files:
        print(f"错误文件列表: {error_files[:10]}{'...' if len(error_files) > 10 else ''}")
    
    return all_activity_ranges

# 执行处理
print("开始处理所有骨架文件，获取每个人的活动范围...")

# 根据CPU核心数自动设置线程数，通常设置为核心数或核心数的2倍
import multiprocessing
cpu_count = multiprocessing.cpu_count()
max_workers = min(cpu_count * 2, 16)  # 最多使用16个线程避免过度并发

print(f"检测到 {cpu_count} 个CPU核心，将使用 {max_workers} 个线程")

activity_ranges = process_all_skeleton_files(max_workers=max_workers)

# 保存结果到文件
np.save("person_activity_ranges.npy", activity_ranges, allow_pickle=True)

print(f"\n结果已保存为 person_activity_ranges.npy (npy格式)")
print(f"数据结构说明:")
print(f"- 外层key: 文件名 (如 'S001C001P001R001A001.skeleton.npy')")
print(f"- 内层key: 人体编号 (如 'body0', 'body1')")
print(f"- 值: 包含 min_x, min_y, max_x, max_y, width, height, skeleton_data 的字典")

# 显示一个示例
if activity_ranges:
    sample_file = list(activity_ranges.keys())[0]
    print(f"\n示例数据结构 ({sample_file}):")
    for body_key, body_data in activity_ranges[sample_file].items():
        print(f"  {body_key}: 宽度={body_data['width']:.1f}, 高度={body_data['height']:.1f}, 骨架数据形状={body_data['skeleton_data'].shape}")

开始处理所有骨架文件，获取每个人的活动范围...
检测到 64 个CPU核心，将使用 16 个线程
总共需要处理 56579 个文件
使用 16 个线程进行并行处理


处理骨架文件: 100%|██████████| 56579/56579 [00:34<00:00, 1659.39it/s]



处理完成！
成功处理文件: 56579 个
检测到的总人数: 68589 个
错误文件: 0 个

结果已保存为 person_activity_ranges.npy (npy格式)
数据结构说明:
- 外层key: 文件名 (如 'S001C001P001R001A001.skeleton.npy')
- 内层key: 人体编号 (如 'body0', 'body1')
- 值: 包含 min_x, min_y, max_x, max_y, width, height, skeleton_data 的字典

示例数据结构 (S001C001P001R002A029.skeleton.npy):
  body0: 宽度=112.2, 高度=373.0, 骨架数据形状=(112, 25, 2)


In [2]:
# 统计分析和数据概况
def analyze_activity_data(activity_ranges):
    """
    分析活动范围数据，提供基本统计信息
    """
    if not activity_ranges:
        print("没有数据可分析")
        return
    
    # 收集所有身体的统计数据
    all_widths = []
    all_heights = []
    file_body_counts = []
    
    for file_name, file_data in activity_ranges.items():
        file_body_counts.append(len(file_data))
        for body_key, body_data in file_data.items():
            # 检查数据有效性
            if (not np.isnan(body_data['width']) and not np.isnan(body_data['height']) and 
                body_data['width'] > 0 and body_data['height'] > 0):
                all_widths.append(body_data['width'])
                all_heights.append(body_data['height'])
    
    print("=== 活动范围数据统计 ===")
    print(f"总文件数: {len(activity_ranges)}")
    print(f"总人数: {len(all_widths)} (有效数据)")
    print(f"每个文件平均人数: {np.mean(file_body_counts):.1f}")
    
    if all_widths:
        print(f"\n宽度统计:")
        print(f"  平均值: {np.mean(all_widths):.1f} pixels")
        print(f"  范围: {np.min(all_widths):.1f} - {np.max(all_widths):.1f} pixels")
        
        print(f"\n高度统计:")
        print(f"  平均值: {np.mean(all_heights):.1f} pixels") 
        print(f"  范围: {np.min(all_heights):.1f} - {np.max(all_heights):.1f} pixels")
    else:
        print("\n没有有效的数据进行统计")

def show_sample_data(activity_ranges, num_samples=2):
    """
    显示一些示例数据
    """
    print(f"\n=== 示例数据 (前{num_samples}个文件) ===")
    for i, (file_name, file_data) in enumerate(list(activity_ranges.items())[:num_samples]):
        print(f"\n文件: {file_name}")
        print(f"  包含 {len(file_data)} 个人")
        for body_key, body_data in file_data.items():
            skeleton_shape = body_data['skeleton_data'].shape
            print(f"    {body_key}: 宽度={body_data['width']:.0f}, 高度={body_data['height']:.0f}, 骨架形状={skeleton_shape}")

# 运行统计分析
if activity_ranges:
    analyze_activity_data(activity_ranges)
    show_sample_data(activity_ranges)
else:
    print("没有成功处理的数据")

=== 活动范围数据统计 ===
总文件数: 56579
总人数: 67672 (有效数据)
每个文件平均人数: 1.2

宽度统计:
  平均值: 245.6 pixels
  范围: 40.8 - 1836.2 pixels

高度统计:
  平均值: 484.8 pixels
  范围: 92.2 - 1399.5 pixels

=== 示例数据 (前2个文件) ===

文件: S001C001P001R002A029.skeleton.npy
  包含 1 个人
    body0: 宽度=112, 高度=373, 骨架形状=(112, 25, 2)

文件: S001C001P003R001A036.skeleton.npy
  包含 1 个人
    body0: 宽度=146, 高度=399, 骨架形状=(79, 25, 2)


In [1]:
import numpy as np

# 加载joblib格式的处理后数据
activity_ranges = np.load("person_activity_ranges.npy", allow_pickle=True).item()

print(f"成功加载数据:")
print(f"- 文件数: {len(activity_ranges)}")
print(f"- 总人数: {sum(len(file_data) for file_data in activity_ranges.values())}")

# 显示前20个示例
num_samples = 20
print(f"\n前{num_samples}个示例数据:")
for i, (sample_file, file_data) in enumerate(list(activity_ranges.items())[:num_samples]):
    print(f"\n示例 {i+1}: {sample_file}")
    for body_key, body_data in file_data.items():
        print(f"  {body_key}: 宽度={body_data['width']:.1f}, 高度={body_data['height']:.1f}, 左上角点=[{body_data['min_x']}, {body_data['min_y']}], 骨架数据形状={body_data['skeleton_data']}")

成功加载数据:
- 文件数: 56579
- 总人数: 68589

前20个示例数据:

示例 1: S001C001P001R002A029.skeleton.npy
  body0: 宽度=112.2, 高度=373.0, 左上角点=[972.4461, 336.0972], 骨架数据形状=[[[1019.08    513.3568]
  [1018.021   442.3673]
  [1016.886   369.6843]
  ...
  [1049.643   409.111 ]
  [1052.552   407.3719]
  [1053.241   411.6967]]

 [[1019.022   513.406 ]
  [1017.948   442.3774]
  [1016.784   369.6791]
  ...
  [1048.993   408.8485]
  [1051.135   407.3654]
  [1051.077   415.0333]]

 [[1019.254   513.5101]
  [1017.954   442.3824]
  [1016.616   369.6282]
  ...
  [1048.103   409.5424]
  [1041.17    413.494 ]
  [1032.894   423.7254]]

 ...

 [[1017.991   515.688 ]
  [1023.342   447.1503]
  [1029.294   376.5776]
  ...
  [1055.469   422.3254]
  [1062.728   414.3186]
  [1075.493   410.3575]]

 [[1018.381   516.1919]
  [1023.506   447.256 ]
  [1029.332   376.5272]
  ...
  [1054.7     420.4832]
  [1060.901   415.3172]
  [1076.32    412.0851]]

 [[1018.602   516.0767]
  [1023.614   447.3451]
  [1029.336   376.9071]
  ...
  [1054

运行到这转到data_analysis.ipynb继续分析数据